In [1]:
import yaml
from torch.utils.data import DataLoader
import argparse

from GeospatialFM.data import get_datasets
from GeospatialFM.models import *
# from utils import load_config
from torchgeo.samplers import RandomGeoSampler
from matplotlib import pyplot as plt

from transformers import TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup
from GeospatialFM.utils import *
from GeospatialFM.data import *
from GeospatialFM.models import *

from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import numpy as np
from torch.utils.data import ConcatDataset
import segmentation_models_pytorch as smp

%load_ext autoreload
%autoreload 2

/home/haozhesi/anaconda3/envs/sat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0')


In [3]:
args = {'exp_name': None,
        'config_file': 'GeospatialFM/configs/crop.yaml',
        'opts': None, 
        'save_config': False}
args = argparse.Namespace(**args)
args.debug = True
args

Namespace(exp_name=None, config_file='GeospatialFM/configs/crop.yaml', opts=None, save_config=False, debug=True)

In [4]:
cfg, _ = setup(args)

In [5]:
model = CustomCROP(cfg['MODEL'], cfg['SAR_MODEL'], output_dict=True)

In [6]:
model

CustomCROP(
  (visual): ResNet(
    (conv1): Conv2d(13, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, m

In [7]:
train_ds, val_ds, test_ds = get_datasets(cfg['DATASET'])
dataloader = DataLoader(train_ds, batch_size=256, shuffle=True, num_workers=8)
lr = 1e-4
wd = 0
epochs = 1
steps = len(dataloader) * epochs

In [8]:
len(dataloader.dataset)

393418

In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=steps, eta_min=1e-7)
loss = CustomCropLoss()

In [10]:
args = dict(
    device = 0,
    precision = None,
    accum_freq = 1,
    grad_clip_norm = None,
    log_every_n_steps = 10,
    wandb = False,
    batch_size = 256,
)
args = argparse.Namespace(**args)
args

Namespace(device=0, precision=None, accum_freq=1, grad_clip_norm=None, log_every_n_steps=10, wandb=False, batch_size=256)

In [11]:
sample = next(iter(dataloader)) 

In [12]:
image, radar, label = sample['image'], sample['radar'], sample['label']
output_dict = model(image, radar)

In [13]:
losses = loss(**output_dict, gt_labels=label, output_dict=True)

In [14]:
losses

{'contrastive_loss': tensor(5.7446, grad_fn=<DivBackward0>),
 'image_loss': tensor(2.9434, grad_fn=<NllLossBackward0>),
 'radar_loss': tensor(2.9503, grad_fn=<NllLossBackward0>),
 'total_loss': tensor(11.6382, grad_fn=<AddBackward0>)}